# Piano to Midi Transcription
## Process outline:
    1. Load data
    2. Preprocess data (remove left channel (piano in aebersold is only on right channel) - optional)
    3. Extract piano source with DEMUCS source separator
    4. Transcribe piano with Piano Transcription Inference (PTI)-model to MIDI
    (5. Listen to MIDI-files in ./evaluation.ipynb)


### Install and load Packages

In [1]:
#!conda install mamba --yes
#!mamba install ffmpeg==4.1.3 --yes
# needed for torchaudio to be able to handle mp3 

### Troubleshooting / make necessary installations (should not be required if used provided requirements.txt to create environment)
#!pip install piano_transcription_inference
#!pip install pyfluidsynth
#!conda install mamba --yes # mamba makes installations in conda faster
#!mamba install numpy<1.24.0 --yes # pretty midi needs an older numpy version, use conda or pip if you dont have mamba
#!conda install librosa==0.9.2 -c conda-forge --yes 
# needed version of librosa, restart kernel if error does not go away

### Restart kernel after installing packages! ###

In [1]:
from piano_transcription_utils import *
from IPython.display import Audio
import soundfile as sf
import numpy as np
import scipy
import librosa
from glob import glob
import os
import logging
import shutil

#demucs imports
import torch
import torchaudio
from torchaudio.transforms import Fade
from torchaudio.utils import download_asset

#PTI imports
#PTI will install files locally when first run
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio


DEBUG:matplotlib data path: c:\Users\Sebi\anaconda3\envs\nlp\envs\prac\Lib\site-packages\matplotlib\mpl-data
DEBUG:CONFIGDIR=C:\Users\Sebi\.matplotlib
DEBUG:interactive is False
DEBUG:platform is win32
DEBUG:CACHEDIR=C:\Users\Sebi\.matplotlib
DEBUG:Using fontManager instance from C:\Users\Sebi\.matplotlib\fontlist-v330.json


### Setup 

In [2]:
# configure logger
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

#specify global variables
PATH_TO_FILES = os.path.join("audio") #relative filepath
OUT_PATH = os.path.join("audio","out")  #relative filepath
RECURSIVE = True # set to <True> to include subfolders
PAN = "1"  # str:"0" for left channel, str:"1" for right channel (piano in aebersold), leave <None> for no panning (e.g already separated Filosax data)
FILETYPE = ["wav","mp3"] #specify filetype(s) e.g. ["wav","mp3"] , have to be supported by torchaudio
DEVICE = "cuda" # make it str:"cuda" if available
TRIM = False # set to <True> if you want to trim audio (shorter waiting times for testing)
START = 0 # set starting point in seconds for trimming, default is 0
STOP = 0 # set end point for trimming, set to int:0 if no end trimming.
KEEP_DEMUCS = False # set to <False> if you don't want demucs-separated sources saved locally
FORMATTER = ":03d" # set to str:"02d" for max 100, str:"03d" for max 1000 files if you want file formatting, else leave <None>

In [3]:
# initialize demucs
initialize_demucs()

INFO:Initialized Demucs successfully


### Load files

In [4]:
# get input audio files
f_list = get_files(PATH_TO_FILES,OUT_PATH,FILETYPE,RECURSIVE)
'''
### Uncomment this if you want to filter the files by a given substring
substring = "Piano"
valid = []
for i in range(len(f_list)):
    if f_list[i].find(substring) != -1:
        valid.append(f_list[i])
    else:
        continue
f_list = valid
'''
f_list # show found files


INFO:Created output folders.
INFO:Found 13 audiofiles.


['audio\\Vol. 16 Turnaround & Cycles\\1-01  Old Turnaround No.1.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-02 Turnarounds n.2.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-03 Turnarounds n.3.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-04 Turnarounds n.4.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-05 II-V7-I.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-06 Hale Bop.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-07 Cycles N.1(2 bars ea).mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-08 Cycles N.2(1 bar ea.).mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-09 Cycles N.3(2 beats ea).mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-10 Cycles N.4- fast tempo.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-11 Cycles N.5-fast tempo.mp3',
 'audio\\Vol. 16 Turnaround & Cycles\\1-12 V7+9-1(1 bar ea.).mp3',
 'audio\\out\\demucs\\1-01  Old Turnaround No.1001.wav']

### Preprocess and Transcribe audio

In [5]:
# start transcription
logging.getLogger().setLevel(logging.INFO)
for i,f in enumerate(f_list):
    # configure out-file formatter
    fname = os.path.basename(f)[:-4]
    if FORMATTER:
        fname_formatted = ("{"+FORMATTER+"}").format(i+1)
    else:
        fname_formatted = fname
    n = len(f_list)
    logging.info('#'*99)
    logging.info(f"Processing file {i+1}/{n}: {fname} ...")
    # pan audio
    wave,sr = load_and_pan(f,pan=PAN,trim=TRIM,start=START,stop=STOP)
    # separate piano source
    source_p, sr = separate_audio(file=wave,sr=sr,filename=f'{fname_formatted}.wav',source='piano',device = DEVICE, save_to_disk = True,out_path=OUT_PATH)    
    # make transcription
    logging.info(os.path.join(os.path.dirname(f),'demucs',f"{fname}.wav"))    
    make_transcription(os.path.join(OUT_PATH,'demucs',f'{fname_formatted}.wav'),os.path.join(OUT_PATH,f'{fname_formatted}.mid'),DEVICE)

# delete demucs files if specified
logging.info('#'*99)
if not KEEP_DEMUCS:
    logging.info(f"{os.path.join(OUT_PATH,'demucs')} and all its content will be removed, are you sure? y/n ")
    if input() == "y":
        shutil.rmtree(os.path.join(OUT_PATH,'demucs'))
    else:
        logging.info("Aborted removal.")
    
logging.info("Finished.")
    
    
    

INFO:###################################################################################################
INFO:Processing file 1/13: 1-01  Old Turnaround No.1 ...
INFO:The local file (C:\Users\Sebi\.cache\torch\hub\torchaudio\models\hdemucs_high_trained.pt) exists. Skipping the download.
INFO:Separating track from local data ...
INFO:...finished.
INFO:Saved file to: audio\out\demucs\1-01  Old Turnaround No.1001.wav .
INFO:audio\Vol. 16 Turnaround & Cycles\demucs\1-01  Old Turnaround No.1.wav
c:\Users\Sebi\anaconda3\envs\nlp\envs\prac\Lib\site-packages\piano_transcription_inference\utilities.py:556: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y = librosa.core.audio.resample(y, sr_native, sr, res_type=res_type)


Checkpoint path: D:\Sebastian\Uni\AI\BAC\project-ai\piano_transcription_inference_data\CRNN_note_F1=0.9677_pedal_F1=0.9186.pth
Using cuda for inference.
GPU number: 1
Segment 0 / 11
Segment 1 / 11
Segment 2 / 11
Segment 3 / 11
Segment 4 / 11
Segment 5 / 11
Segment 6 / 11
Segment 7 / 11
Segment 8 / 11
Segment 9 / 11
Segment 10 / 11
Segment 11 / 11


INFO:Transcribed 1-01  Old Turnaround No.1001.mid and saved to disk.
INFO:###################################################################################################
INFO:Processing file 2/13: 1-02 Turnarounds n.2 ...


Write out to audio\out\1-01  Old Turnaround No.1001.mid


INFO:The local file (C:\Users\Sebi\.cache\torch\hub\torchaudio\models\hdemucs_high_trained.pt) exists. Skipping the download.
INFO:Separating track from local data ...
INFO:...finished.
INFO:Saved file to: audio\out\demucs\1-02 Turnarounds n.2002.wav .
INFO:audio\Vol. 16 Turnaround & Cycles\demucs\1-02 Turnarounds n.2.wav
c:\Users\Sebi\anaconda3\envs\nlp\envs\prac\Lib\site-packages\piano_transcription_inference\utilities.py:556: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y = librosa.core.audio.resample(y, sr_native, sr, res_type=res_type)


...

INFO:The local file (C:\Users\Sebi\.cache\torch\hub\torchaudio\models\hdemucs_high_trained.pt) exists. Skipping the download.
INFO:Separating track from local data ...
INFO:...finished.
INFO:Saved file to: audio\out\demucs\1-01  Old Turnaround No.1001013.wav .
INFO:audio\out\demucs\demucs\1-01  Old Turnaround No.1001.wav


Checkpoint path: D:\Sebastian\Uni\AI\BAC\project-ai\piano_transcription_inference_data\CRNN_note_F1=0.9677_pedal_F1=0.9186.pth
Using cuda for inference.
GPU number: 1
Segment 0 / 11
Segment 1 / 11
Segment 2 / 11
Segment 3 / 11
Segment 4 / 11
Segment 5 / 11
Segment 6 / 11
Segment 7 / 11
Segment 8 / 11
Segment 9 / 11
Segment 10 / 11
Segment 11 / 11


INFO:Transcribed 1-01  Old Turnaround No.1001013.mid and saved to disk.
INFO:###################################################################################################
INFO:audio\out\demucs and all its content will be removed, are you sure? y/n 


Write out to audio\out\1-01  Old Turnaround No.1001013.mid


INFO:Aborted removal.
INFO:Finished.


In [6]:
# put <y> or <n> to make sure nothing is deleted involuntarily!